In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Base URL
base_url = 'https://deals.jumia.ci/terrains-parcelles'

# Initialize an empty DataFrame to store the data
df = pd.DataFrame()

while True:
    # Generate the URL for the current page
    url = base_url

    # Send a GET request
    r = requests.get(url)
    
    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # Find all articles on the page
    biens = soup.find_all('div', class_='text-area')
    
    # If the page does not contain articles, stop the loop
    if not biens:
        break
    
    # List to hold the scraped data
    immobiliers = []
    
    # Loop through each article
        
    for bien in biens:
        # Extract the data and clean it up
        titre = bien.find("div", {"class":"announcement-infos"}).find("a", {"class":"post-link post-vip"}).find("span").text
        localisation = bien.find("div", {"class":"announcement-infos"}).find("span", {"class":"address"}).text.replace('\n', '').strip()
        prix = bien.find("div", {"class":"price-date"}).find("span", {"class":"price"}).text.strip().replace('\xa0', '')

        # Find the URL for the individual bien
        individual_url = "https://deals.jumia.ci" + bien.find("div", {"class":"announcement-infos"}).find("a", {"class":"post-link post-vip"}).get('href')

        # Send a GET request to the individual URL
        r_individual = requests.get(individual_url)

        # Parse the content with BeautifulSoup
        soup_individual = BeautifulSoup(r_individual.text, 'html.parser')

        # Find the details on the page
        details = soup_individual.find('div', class_='new-attr-style')

        # Extract the number of pieces and the area if they exist
        if details:
            h3_elements = details.find_all('h3')
            nbre_pieces = h3_elements[0].find('span').text if len(h3_elements) > 0 else None
            superficie = h3_elements[1].find('span').text if len(h3_elements) > 1 else None
        else:
            nbre_pieces = None
            superficie = None

        # Save the data in a dictionary
        data = {'Title': titre, 'Location': localisation, 'Price': prix, 'Number of Pieces': nbre_pieces, 'Area': superficie}

        # Append the data to the list
        immobiliers.append(data)
        print(data)
    
    # Convert the list into a DataFrame and append it to the main DataFrame
    df = df.append(pd.DataFrame(immobiliers), ignore_index=True)

    # Find the 'Next' button
    next_button = soup.find('li', class_='next')

    # If the 'Next' button is disabled, stop the loop
    if 'disable' in next_button.get('class', []):
        break

    # Otherwise, update the base_url to the next page
    else:
        base_url = 'https://deals.jumia.ci' + next_button.find('a').get('href')

    # Pause for 1 second
    time.sleep(1)

# Save the DataFrame to an Excel file
df.to_excel('immobiliers.xlsx', index=False)


In [4]:
import os

os.getcwd()

'C:\\Users\\HP\\Downloads\\projet\\Untitled Folder'